In [50]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from datetime import datetime
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [51]:
df = pd.read_excel('/Users/roshan/Downloads/PVADATA.xlsx', sheet_name = 'Raw Data_06.26.23')
df.dropna(axis = 1, how="all", inplace=True); #dropping columns that have all null entries

In [52]:
df.drop(['Location Latitude',
         'Location Longitude',
         'HMIS ID',
         'Start Date',
         'End Date',
         'Response Type',
         'IP Address',
         'Progress',
         'Duration (in seconds)',
         'Finished',
         'Recorded Date',
         'Response ID',
         'Distribution Channel',
         'User Language',
         'Consent',
         'Case manager completing Place Value assessment - Selected Choice',
         'Case manager completing Place Value assessment - Other - Text',
         'Case Manager Email Address',
         'Homeless before med',
         'Living Situation Details',
         'Homeless before corrections',
         'Confirm',
         'Embedded Household Score',
         'Score',
         'Scored Score',
         'Final Score'], axis = 1, inplace = True)

In [53]:
def replace_columns(df, old_colm, new_colm, new_values: list ):
  df.rename(columns={old_colm: new_colm}, inplace=True)
  df[new_colm] = new_values


In [54]:
# Converting the DOB Date of Birth columns to Age
date_of_birth_list = df['DOB']

# Get the current date
current_date = datetime.now()

# Create an empty list to store the ages
age_list = []

# Calculate age for each date of birth
for dob in date_of_birth_list:
    dob = str(dob)
    # Convert the date of birth string to a datetime object
    dob_date = datetime.strptime(dob, '%Y-%m-%d %H:%M:%S')

    # Calculate the difference between current date and date of birth
    age = (current_date - dob_date).days // 365

    # Append the age to the age list
    age_list.append(age)

replace_columns(df, 'DOB', 'Age', age_list)

> We notice that, for the gender column, the dataset has two skip options. Therefore, we are merging the two options into one below.


In [55]:
gender_col_entries = df['How do you identify your gender?']
two_skips_merged = []
for val in gender_col_entries:
  if val == 'skip':
    val = 'Skip this question'
  two_skips_merged.append(val)

df['How do you identify your gender?'] = two_skips_merged

In [56]:
import math
import plotly.graph_objs as go
from plotly.subplots import make_subplots

class Plotter:
    """
    Base class for creating plotter objects.

    Parameters:
        df (DataFrame): The input DataFrame to plot.
        num_cols (int): Number of columns for the subplot grid.
    """

    def __init__(self, df, num_cols=3):
        """
        Initializes a Plotter object.

        Args:
            df (DataFrame): The input DataFrame to plot.
            num_cols (int): Number of columns for the subplot grid.
        """
        self.df = df
        self.num_cols = num_cols
        self.num_plots = len(df.columns) if df is not None else 0
        self.num_rows = math.ceil(self.num_plots / self.num_cols)

    # Function to truncate column names
    @staticmethod
    def truncate_string(text, max_length):
        return text[:max_length - 3] + "..." if len(text) > max_length else text

    def plot_plots(self):
        """
        Plots the subplots of the DataFrame columns.
        """
        if self.num_plots == 0:
            print("No data to plot.")
            return

        # Create a list to store column names with more than one unique value
        valid_columns = []

        # Check if the column has more than one unique value
        for column in self.df.columns:
            if self.df[column].nunique() > 1:
                valid_columns.append(column)

        # Create the subplots grid
        fig = make_subplots(rows=self.num_rows, cols=self.num_cols, subplot_titles=valid_columns)

        # Loop through each valid column
        for i, column in enumerate(valid_columns):
            numeric_column_list = self.df._get_numeric_data().columns
            if column in numeric_column_list:
                self.kdeplot(column, fig, i + 1)
            else:
                self.histplot(column, fig, i + 1)

        # Update subplot layout settings
        fig.update_layout(
            showlegend=False,
            height=400 * self.num_rows,
            width=800,
            title_font=dict(size=12),
            font=dict(size=8),  # Decrease the font size
            hovermode='x unified',
            # title_text="Plotter in Plotly",  # Set the main title
            margin=dict(t=100, b=10)  # Adjust the top and bottom margins
        )

        # Update axis layout settings
        fig.update_xaxes(showticklabels=False)
        fig.update_yaxes(showticklabels=True)

        # Truncate and adjust subplot titles
        max_title_length = 20  # Maximum number of characters for each title
        for i, subplot_title in enumerate(fig.layout.annotations):
            # Set the full title as hover text
            subplot_title.hovertext = subplot_title.text

            # Truncate the title if needed
            truncated_title = self.truncate_string(subplot_title.text, max_title_length)
            fig.layout.annotations[i].text = truncated_title

        return fig

    def histplot(self, column, fig, subplot_index, order_='ascending'):
        """
        Plots a histogram for a single column.

        Args:
            column (str): The name of the column to plot.
            fig (plotly.graph_objs.Figure): The Plotly figure object.
            subplot_index (int): The index of the subplot.
        """
        # Create the histogram trace
        hist_trace = go.Histogram(x=self.df[column], name=column)

        # Add the trace to the subplot
        fig.add_trace(hist_trace, row=(subplot_index - 1) // self.num_cols + 1, col=(subplot_index - 1) % self.num_cols + 1)

    def kdeplot(self, column, fig, subplot_index):
        """
        Plots a kernel density estimation plot for a single column.

        Args:
            column (str): The name of the column to plot.
            fig (plotly.graph_objs.Figure): The Plotly figure object.
            subplot_index (int): The index of the subplot.
        """
        # Create the KDE plot trace as a violin plot
        kde_trace = go.Violin(
            y=self.df[column],
            name=column,
            box_visible=True,
            meanline_visible=True,
            fillcolor='lightseagreen',
            line_color='darkgreen'
        )

        # Add the trace to the subplot
        fig.add_trace(kde_trace, row=(subplot_index - 1) // self.num_cols + 1, col=(subplot_index - 1) % self.num_cols + 1)


plotter = Plotter(df, num_cols=3)
plotter.plot_plots()


In [57]:
# Create a dictionary to map the column names
import json

with open('columns_mapping.json') as f:
    columns_mapping = json.load(f)

# Rename the columns

df.rename(columns=columns_mapping, inplace=True)

In [58]:
df['gender'].value_counts()

FEMALE                   516
MALE                     374
Skip this question         2
GENDER NON-CONFORMING      1
Name: gender, dtype: int64

In [59]:
def white_BIPOC(row):
  race = row['race']
  if race == 'White':
    return 'White'
  else:
    return 'BIPOC'

df['race'] = df.apply(white_BIPOC, axis=1)

df['race'].value_counts()


BIPOC    457
White    436
Name: race, dtype: int64

In [60]:
def male_female_other(row):
  gender = row['gender']
  if gender == 'Skip this question' or gender == 'GENDER NON-CONFORMING':
    return 'OTHER'
  else:
    return gender

df['gender'] = df.apply(male_female_other, axis=1)

df['gender'].value_counts()

FEMALE    516
MALE      374
OTHER       3
Name: gender, dtype: int64

In [61]:
def age_map(age):
    if age < 1:
        return None
    elif age < 18:
        return 'Under 18'
    elif age < 25:
        return '18-24'
    elif age < 35:
        return '25-34'
    elif age < 45:
        return '35-44'
    elif age < 55:
        return '45-54'
    elif age < 65:
        return '55-64'
    else:
        return '64 and over'

df['AGE_GROUP'] = df['age'].apply(age_map)

In [62]:
print(df['race'].value_counts(), end="\n\n")
print(df['gender'].value_counts())

BIPOC    457
White    436
Name: race, dtype: int64

FEMALE    516
MALE      374
OTHER       3
Name: gender, dtype: int64


## Adding new columns Male, Female, BIPOC, White

In [63]:
df['sex_Male'] = df.apply(lambda x: 1 if x['gender'] == 'MALE' else 0, axis=1)
df['sex_Female'] = df.apply(lambda x: 1 if x['gender'] == 'FEMALE' else 0, axis = 1)
df['race_BIPOC'] = df.apply(lambda x: 1 if x['race'] == 'BIPOC' else 0, axis = 1)
df['race_White'] = df.apply(lambda x: 1 if x['race'] == 'White' else 0, axis = 1)

In [64]:
df.head()

,diversion_attempt?,county,age,race,hispanic/latino?,gender,living_situation,lived_in_uninhabitable_place?,lived_in_emergency_shelter?,lived_in_transitional_housing?,...,living_situation_score,health_scale_score,LOT_homeless_score,risks_and_barriers_score,pva_priority_score,AGE_GROUP,sex_Male,sex_Female,race_BIPOC,race_White
0,Yes,Hamilton,24,BIPOC,Non Hispanic / Non Latino,FEMALE,"Outdoors, car, vacant building, or other unshe...",NaN,NaN,NaN,...,5,1,1,2,11215,18-24,0,1,1,0
1,Yes,Hamilton,38,BIPOC,Non Hispanic / Non Latino,MALE,Hotel or temporary rental requiring ongoing pa...,"Yes, I've spent one or more nights in a place ...",NaN,NaN,...,4,8,4,0,48024,35-44,1,0,1,0
2,Yes,Hamilton,40,BIPOC,Non Hispanic / Non Latino,FEMALE,"Outdoors, car, vacant building, or other unshe...",NaN,NaN,NaN,...,3,4,2,3,24313,35-44,0,1,1,0
3,Yes,Hamilton,41,BIPOC,Non Hispanic / Non Latino,FEMALE,Emergency shelter (including hotel paid for by...,NaN,NaN,NaN,...,3,4,1,2,14213,35-44,0,1,1,0
4,Yes,Hamilton,26,BIPOC,Non Hispanic / Non Latino,FEMALE,Emergency shelter (including hotel paid for by...,NaN,NaN,NaN,...,4,5,1,5,15514,25-34,0,1,1,0
